In [1]:
# import os
# import glob
# import pandas as pd

# # Set the directory path where the files are located
# directory = "d:\\DataExport\\"

# # Get a list of all CSV files in the directory
# file_list = glob.glob(os.path.join(directory, "*.csv"))

# # Create an empty DataFrame to store the merged data
# merged_data = pd.DataFrame()

# # Iterate over each file and merge the data
# for file in file_list:
#     df = pd.read_csv(file)
#     merged_data = pd.concat([merged_data, df])

# # Write the merged data to a new CSV file
# merged_data.to_csv("stock_data.csv", index=False)


In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

# URL of the page you want to scrape
url = 'https://fireant.vn/ma-chung-khoan/VN30F1M'

# Create a new instance of the Firefox driver
driver = webdriver.Firefox()

# Go to the URL
driver.get(url)

# Get the HTML of the page
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table with the data (replace 'table-id' with the actual ID or attributes of the table)
table = soup.find('table', attrs={'id': 'table-id'})

# Get the headers of the table
headers = [header.text for header in table.find_all('th')]

# Get the rows of the table
rows = [[cell.text for cell in row.find_all('td')] for row in table.find_all('tr')]

# Create a DataFrame from the headers and rows
df = pd.DataFrame(rows, columns=headers)

# Write the DataFrame to a CSV file
df.to_csv('VN30F1M.csv', index=False)

# Close the browser
driver.quit()

ModuleNotFoundError: No module named 'selenium'

In [2]:
import pandas as pd
import configparser

# Read the value from the config.ini file
config = configparser.ConfigParser()
config.read('config.ini')
try:
    file_path = config.get('data', 'file_path1')
    # Read the data from the csv file
    df = pd.read_csv(file_path, engine='pyarrow')

except FileNotFoundError as e:
    file_path = config.get('data', 'file_path2')
    # Read the data from the csv file
    df = pd.read_csv(file_path, engine='pyarrow')
except:
    print('Error reading the file path from the config file')

print(df.head())

    Symbol    Date   Open   High    Low  Close  Volume
0  VN30F1M  170810  757.9  757.9  745.5  745.9     367
1  VN30F1M  170811  746.0  746.4  743.4  745.7     510
2  VN30F1M  170814  745.9  748.9  745.7  748.9     707
3  VN30F1M  170815  750.3  750.9  748.0  748.0     946
4  VN30F1M  170816  747.7  748.4  745.1  748.1    1065


In [3]:
# Drop the rows with missing values
df.dropna(inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474 entries, 0 to 1473
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Symbol  1474 non-null   object 
 1   Date    1474 non-null   int64  
 2   Open    1474 non-null   float64
 3   High    1474 non-null   float64
 4   Low     1474 non-null   float64
 5   Close   1474 non-null   float64
 6   Volume  1474 non-null   int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 80.7+ KB
None


In [4]:
import pandas as pd
import numpy as np

# Assuming your dataframe is named "df"
# Convert the "Date" column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%y%m%d')

# Map the day of the week to corresponding labels
day_labels = ['Mo', 'Tu', 'We', 'Th', 'Fr', 'Sa', 'Su']
df['DoW'] = df['Date'].dt.dayofweek.map(lambda x: day_labels[x])

# Check if the date is the 3rd Thursday of the month and label it as "Th-expired"
DoWExp = config.get('derivatives', 'DoWExp')

df['DoW'] = np.where((df['Date'].dt.weekday == 3) & (df['Date'].dt.day <= 21) & (df['Date'].dt.day >= 15), DoWExp + '-exp', df['DoW'])

# Revert the "Date" column back to the original format
df['Date'] = df['Date'].dt.strftime('%y%m%d')

# Print the updated dataframe
print(df)


       Symbol    Date    Open    High     Low   Close  Volume DoW
0     VN30F1M  170810   757.9   757.9   745.5   745.9     367  Th
1     VN30F1M  170811   746.0   746.4   743.4   745.7     510  Fr
2     VN30F1M  170814   745.9   748.9   745.7   748.9     707  Mo
3     VN30F1M  170815   750.3   750.9   748.0   748.0     946  Tu
4     VN30F1M  170816   747.7   748.4   745.1   748.1    1065  We
...       ...     ...     ...     ...     ...     ...     ...  ..
1469  VN30F1M  230627  1125.5  1125.9  1120.8  1125.2  132592  Tu
1470  VN30F1M  230628  1127.0  1136.0  1126.5  1132.4  139703  We
1471  VN30F1M  230629  1132.2  1132.5  1115.0  1115.0  189512  Th
1472  VN30F1M  230630  1115.6  1119.9  1114.4  1117.3  151936  Fr
1473  VN30F1M  230703  1120.7  1122.2  1115.8  1117.5  130472  Mo

[1474 rows x 8 columns]


In [5]:
import numpy as np

# Create the FO column
df['Adj_Open'] = np.where(df['Open'] > df['Close'].shift(1), 'OU', 'OD')

# Create the FC column
df['Adj_Close'] = np.where(df['Close'] > df['Close'].shift(1), 'CU', 'CD')

# Calculate the point of High from Previous Close
df['HPC'] = (df['High'] - df['Close'].shift(1))

# Calculate the point of Low from Previous Close
df['LPC'] = (df['Low'] - df['Close'].shift(1))

# Calculate the point of High from Open
df['HFO'] = (df['High'] - df['Open'])

# Calculate the point of Low from Open
df['LFO'] = (df['Low'] - df['Open'])

# Calculate the point of High from Close
df['HFC'] = (df['High'] - df['Close'])

# Calculate the point of Low from Close
df['LFC'] = (df['Low'] - df['Close'])

In [6]:
test_df = df[df['DoW'] == 'Th-exp']

# Count number of CD and CU in test_df
print(test_df['Adj_Close'].value_counts())


# Count number of OU and OD in test_df
print(test_df['Adj_Open'].value_counts())



Adj_Close
CU    36
CD    34
Name: count, dtype: int64
Adj_Open
OD    36
OU    34
Name: count, dtype: int64


In [7]:
# Count number of CD and CU in df, excluding Th-exp
df_excl_th = df[df['DoW'] != (DoWExp + '-exp')]
print(df_excl_th['Adj_Close'].value_counts())



Adj_Close
CU    766
CD    638
Name: count, dtype: int64


In [8]:
import pandas as pd

# Assuming your dataframe is named "df"

# Define the categories
categories = ['CU', 'CD']

# Create a pivot table to calculate the count
pivot_df = pd.pivot_table(df, index='DoW', values='Adj_Open', columns='Adj_Close', aggfunc='count', fill_value=0)

# Calculate the total count
pivot_df['Total'] = pivot_df.sum(axis=1)

# Calculate the percentages
for category in categories:
    pivot_df[f'%{category}'] = pivot_df[category] / pivot_df['Total']

# Reorder the columns
columns = categories + [f'%{category}' for category in categories] + ['Total']
pivot_df = pivot_df.reindex(columns=columns)

# Reindex the rows to match the desired order
day_labels = ['Su', 'Mo', 'Tu', 'We', 'Th',(DoWExp + '-exp'), 'Fr', 'Sa']
pivot_df = pivot_df.reindex(day_labels)

# Drop Total column
pivot_df.drop('Total', axis=1, inplace=True)

pivot_df.dropna(inplace=True)

pivot_df

Adj_Close,CU,CD,%CU,%CD
DoW,,,,
Mo,148.0,140.0,0.513889,0.486111
Tu,169.0,127.0,0.570946,0.429054
We,171.0,124.0,0.579661,0.420339
Th,115.0,113.0,0.504386,0.495614
Th-exp,36.0,34.0,0.514286,0.485714
Fr,163.0,134.0,0.548822,0.451178


In [9]:
import pandas as pd

# Assuming your dataframe is named "df"

# Define the categories
categories = ['OU', 'OD']

# Create a pivot table to calculate the count
pivot_df_1 = pd.pivot_table(df, index='DoW', values='Adj_Close', columns='Adj_Open', aggfunc='count', fill_value=0)

# Calculate the total count
pivot_df_1['Total'] = pivot_df_1.sum(axis=1)

# Calculate the percentages
for category in categories:
    pivot_df_1[f'%{category}'] = pivot_df_1[category] / pivot_df_1['Total']

# Reorder the columns
columns = categories + [f'%{category}' for category in categories] + ['Total']
pivot_df_1 = pivot_df_1.reindex(columns=columns)

# Reindex the rows to match the desired order
day_labels = ['Su', 'Mo', 'Tu', 'We', 'Th',(DoWExp + '-exp'), 'Fr', 'Sa']
pivot_df_1 = pivot_df_1.reindex(day_labels)

# Drop Total column
pivot_df_1.drop('Total', axis=1, inplace=True)

# Drop the rows with missing values
pivot_df_1.dropna(inplace=True)

pivot_df_1

Adj_Open,OU,OD,%OU,%OD
DoW,,,,
Mo,175.0,113.0,0.607639,0.392361
Tu,163.0,133.0,0.550676,0.449324
We,166.0,129.0,0.562712,0.437288
Th,129.0,99.0,0.565789,0.434211
Th-exp,34.0,36.0,0.485714,0.514286
Fr,169.0,128.0,0.569024,0.430976


In [10]:
df

,Symbol,Date,Open,High,Low,Close,Volume,DoW,Adj_Open,Adj_Close,HPC,LPC,HFO,LFO,HFC,LFC
0,VN30F1M,170810,757.9,757.9,745.5,745.9,367,Th,OD,CD,NaN,NaN,0.0,-12.4,12.0,-0.4
1,VN30F1M,170811,746.0,746.4,743.4,745.7,510,Fr,OU,CD,0.5,-2.5,0.4,-2.6,0.7,-2.3
2,VN30F1M,170814,745.9,748.9,745.7,748.9,707,Mo,OU,CU,3.2,0.0,3.0,-0.2,0.0,-3.2
3,VN30F1M,170815,750.3,750.9,748.0,748.0,946,Tu,OU,CD,2.0,-0.9,0.6,-2.3,2.9,0.0
4,VN30F1M,170816,747.7,748.4,745.1,748.1,1065,We,OD,CU,0.4,-2.9,0.7,-2.6,0.3,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469,VN30F1M,230627,1125.5,1125.9,1120.8,1125.2,132592,Tu,OU,CU,2.9,-2.2,0.4,-4.7,0.7,-4.4
1470,VN30F1M,230628,1127.0,1136.0,1126.5,1132.4,139703,We,OU,CU,10.8,1.3,9.0,-0.5,3.6,-5.9
1471,VN30F1M,230629,1132.2,1132.5,1115.0,1115.0,189512,Th,OD,CD,0.1,-17.4,0.3,-17.2,17.5,0.0
1472,VN30F1M,230630,1115.6,1119.9,1114.4,1117.3,151936,Fr,OU,CU,4.9,-0.6,4.3,-1.2,2.6,-2.9


In [11]:
import pandas as pd
import numpy as np

# Create the points dataframe
points_df = pd.DataFrame({'Points': range(1, 15)})

# Calculate the percentage of points >= HFO
points_df['%HFO'] = points_df['Points'].apply(lambda x: (df['HFO'] >= x).mean() * 100)

# Calculate the percentage of points >= abs(LFO)
points_df['%LFO'] = points_df['Points'].apply(lambda x: (np.abs(df['LFO']) >= x).mean() * 100)

# Calculate the percentage of points >= HFC
points_df['%HFC'] = points_df['Points'].apply(lambda x: (df['HFC'] >= x).mean() * 100)

# calculate the percentage of points >= abs(LFC)
points_df['%LFC'] = points_df['Points'].apply(lambda x: (np.abs(df['LFC']) >= x).mean() * 100)

# Calculate the percentage of points >= HPC
points_df['%HPC'] = points_df['Points'].apply(lambda x: (df['HPC'] >= x).mean() * 100)

# Calculate the percentage of points >= abs(LPC)
points_df['%LPC'] = points_df['Points'].apply(lambda x: (np.abs(df['LPC']) >= x).mean() * 100)
# Print the resulting dataframe
print(points_df)


    Points       %HFO       %LFO       %HFC       %LFC       %HPC       %LPC
0        1  87.584803  86.363636  81.071913  84.803256  83.717775  88.466757
1        2  76.797829  75.169607  71.099050  76.594301  77.611940  77.476255
2        3  68.046133  65.943012  62.211669  70.149254  70.556309  69.810041
3        4  60.312076  57.937585  55.698779  63.704206  63.500678  61.668928
4        5  53.527815  51.153324  50.000000  56.173677  57.801900  54.002714
5        6  47.218453  45.454545  44.979647  51.085482  53.459973  47.693351
6        7  41.451832  40.773406  39.959294  44.301221  47.693351  42.672999
7        8  36.567164  36.227951  35.413840  40.230665  42.672999  38.195387
8        9  31.750339  32.700136  32.225237  34.803256  38.263229  34.803256
9       10  28.629579  29.782904  28.968792  31.004071  34.667571  31.139756
10      11  25.983718  26.729986  26.526459  27.679783  31.139756  28.629579
11      12  22.930801  24.830393  23.337856  24.491180  27.747626  26.119403

In [12]:
import pandas as pd

# Method 1: Buying at the close and selling at the open the next day
df_method1 = df.copy()
df_method1['Entry'] = df_method1['Close'].shift()
df_method1['Exit'] = df_method1['Open']
df_method1['Points'] = df_method1['Exit'] - df_method1['Entry']
cumulative_points_method1 = df_method1['Points'].cumsum().iloc[-1]

# Method 2: Shorting at the open plus 2 points and covering at the close
df_method2 = df.copy()
df_method2['Entry'] = df_method2['Open'] + 2
df_method2['Exit'] = df_method2['Close']
df_method2['Points'] = df_method2['Exit'] - df_method2['Entry']
cumulative_points_method2 = df_method2['Points'].cumsum().iloc[-1]

# Method 3: Buying at the open minus 2 points and selling at the close
df_method3 = df.copy()
df_method3['Entry'] = df_method3['Open'] - 2
df_method3['Exit'] = df_method3['Close']
df_method3['Points'] = df_method3['Exit'] - df_method3['Entry']
cumulative_points_method3 = df_method3['Points'].cumsum().iloc[-1]

# Compare the cumulative points for each method
method_points = {
    'Method 1': cumulative_points_method1,
    'Method 2': cumulative_points_method2,
    'Method 3': cumulative_points_method3
}

# Find the method with the most points
best_method = max(method_points, key=method_points.get)
best_points = method_points[best_method]

# Print the results
for method, points in method_points.items():
    print(f"{method}: Cumulative Points = {points}")

print(f"\nBest Method: {best_method}")
print(f"Cumulative Points: {best_points}")


Method 1: Cumulative Points = 577.8
Method 2: Cumulative Points = -3166.2000000000035
Method 3: Cumulative Points = 2729.7999999999956

Best Method: Method 3
Cumulative Points: 2729.7999999999956


In [13]:
df

,Symbol,Date,Open,High,Low,Close,Volume,DoW,Adj_Open,Adj_Close,HPC,LPC,HFO,LFO,HFC,LFC
0,VN30F1M,170810,757.9,757.9,745.5,745.9,367,Th,OD,CD,NaN,NaN,0.0,-12.4,12.0,-0.4
1,VN30F1M,170811,746.0,746.4,743.4,745.7,510,Fr,OU,CD,0.5,-2.5,0.4,-2.6,0.7,-2.3
2,VN30F1M,170814,745.9,748.9,745.7,748.9,707,Mo,OU,CU,3.2,0.0,3.0,-0.2,0.0,-3.2
3,VN30F1M,170815,750.3,750.9,748.0,748.0,946,Tu,OU,CD,2.0,-0.9,0.6,-2.3,2.9,0.0
4,VN30F1M,170816,747.7,748.4,745.1,748.1,1065,We,OD,CU,0.4,-2.9,0.7,-2.6,0.3,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469,VN30F1M,230627,1125.5,1125.9,1120.8,1125.2,132592,Tu,OU,CU,2.9,-2.2,0.4,-4.7,0.7,-4.4
1470,VN30F1M,230628,1127.0,1136.0,1126.5,1132.4,139703,We,OU,CU,10.8,1.3,9.0,-0.5,3.6,-5.9
1471,VN30F1M,230629,1132.2,1132.5,1115.0,1115.0,189512,Th,OD,CD,0.1,-17.4,0.3,-17.2,17.5,0.0
1472,VN30F1M,230630,1115.6,1119.9,1114.4,1117.3,151936,Fr,OU,CU,4.9,-0.6,4.3,-1.2,2.6,-2.9


In [14]:
import pandas as pd

# Convert the Date column to string format
df['Date'] = df['Date'].astype(str)

# Filter the dataframe for the desired date range (2022 and 2023)
df_filtered = df[(df['Date'] >= '220101') & (df['Date'] <= '231231')]

# Method 1: Buying at the close and selling at the open the next day
df_method1 = df_filtered.copy()
df_method1['Entry'] = df_method1['Close'].shift()
df_method1['Exit'] = df_method1['Open']
df_method1['Points'] = df_method1['Exit'] - df_method1['Entry']

# Calculate cumulative points for Method 1
if len(df_method1) > 0:
    cumulative_points_method1 = df_method1['Points'].cumsum().iloc[-1]
else:
    cumulative_points_method1 = 0

# Method 2: Shorting at the open plus 2 points and covering at the close
df_method2 = df_filtered.copy()
df_method2['Entry'] = df_method2['Open'] + 2
df_method2['Exit'] = df_method2['Close']
df_method2['Points'] = df_method2['Exit'] - df_method2['Entry']

# Calculate cumulative points for Method 2
if len(df_method2) > 0:
    cumulative_points_method2 = df_method2['Points'].cumsum().iloc[-1]
else:
    cumulative_points_method2 = 0

# Method 3: Buying at the open minus 2 points and selling at the close
df_method3 = df_filtered.copy()
df_method3['Entry'] = df_method3['Open'] - 2
df_method3['Exit'] = df_method3['Close']
df_method3['Points'] = df_method3['Exit'] - df_method3['Entry']

# Calculate cumulative points for Method 3
if len(df_method3) > 0:
    cumulative_points_method3 = df_method3['Points'].cumsum().iloc[-1]
else:
    cumulative_points_method3 = 0

# Compare the cumulative points for each method
method_points = {
    'Method 1': cumulative_points_method1,
    'Method 2': cumulative_points_method2,
    'Method 3': cumulative_points_method3
}

# Find the method with the most points
best_method = max(method_points, key=method_points.get)
best_points = method_points[best_method]

# Print the results
for method, points in method_points.items():
    print(f"{method}: Cumulative Points = {points}")

print(f"\nBest Method: {best_method}")
print(f"Cumulative Points: {best_points}")


Method 1: Cumulative Points = 35.50000000000273
Method 2: Cumulative Points = -1204.9000000000026
Method 3: Cumulative Points = 279.0999999999972

Best Method: Method 3
Cumulative Points: 279.0999999999972


In [15]:
### METHOD 3
import pandas as pd

# Convert the Date column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%y%m%d')

# Filter the dataframe for the desired date range (2022 and 2023)
df_filtered = df[(df['Date'].dt.year.isin([2022, 2023]))]

# Method 3: Buying at the open minus 2 points and selling at the close
df_method3 = df_filtered.copy()
df_method3['Entry'] = df_method3['Open'] - 3
df_method3['Exit'] = df_method3['Close']
df_method3['Points'] = df_method3['Exit'] - df_method3['Entry']

# Calculate cumulative points for Method 3
cumulative_points_method3 = df_method3['Points'].cumsum()

# Group the data by month and year and calculate the points earned for each month and year
df_method3['Month'] = df_method3['Date'].dt.month
df_method3['Year'] = df_method3['Date'].dt.year
monthly_points = df_method3.groupby(['Year', 'Month'])['Points'].sum()

# Print the points earned each month and year
print("Points Earned Each Month and Year:")
print(monthly_points)


Points Earned Each Month and Year:
Year  Month
2022  1         44.8
      2         35.4
      3         40.3
      4        -36.1
      5        -31.5
      6         34.0
      7         22.1
      8        151.5
      9        -40.4
      10       -71.3
      11        93.4
      12        29.4
2023  1        135.1
      2        -59.0
      3         98.3
      4         17.8
      5         74.9
      6        111.6
      7         -0.2
Name: Points, dtype: float64


In [16]:
# Conver Date in df format YYYY-MM-DD to yymmdd
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d').dt.strftime('%y%m%d')

df

,Symbol,Date,Open,High,Low,Close,Volume,DoW,Adj_Open,Adj_Close,HPC,LPC,HFO,LFO,HFC,LFC
0,VN30F1M,170810,757.9,757.9,745.5,745.9,367,Th,OD,CD,NaN,NaN,0.0,-12.4,12.0,-0.4
1,VN30F1M,170811,746.0,746.4,743.4,745.7,510,Fr,OU,CD,0.5,-2.5,0.4,-2.6,0.7,-2.3
2,VN30F1M,170814,745.9,748.9,745.7,748.9,707,Mo,OU,CU,3.2,0.0,3.0,-0.2,0.0,-3.2
3,VN30F1M,170815,750.3,750.9,748.0,748.0,946,Tu,OU,CD,2.0,-0.9,0.6,-2.3,2.9,0.0
4,VN30F1M,170816,747.7,748.4,745.1,748.1,1065,We,OD,CU,0.4,-2.9,0.7,-2.6,0.3,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469,VN30F1M,230627,1125.5,1125.9,1120.8,1125.2,132592,Tu,OU,CU,2.9,-2.2,0.4,-4.7,0.7,-4.4
1470,VN30F1M,230628,1127.0,1136.0,1126.5,1132.4,139703,We,OU,CU,10.8,1.3,9.0,-0.5,3.6,-5.9
1471,VN30F1M,230629,1132.2,1132.5,1115.0,1115.0,189512,Th,OD,CD,0.1,-17.4,0.3,-17.2,17.5,0.0
1472,VN30F1M,230630,1115.6,1119.9,1114.4,1117.3,151936,Fr,OU,CU,4.9,-0.6,4.3,-1.2,2.6,-2.9


In [17]:
# METHOD TRADING USING MA 10

import pandas as pd

# Calculate MA10
df['MA10'] = df['Close'].rolling(window=10).mean()

# Define trading signals based on MA10
df['Signal'] = 0
df.loc[df['Close'] > df['MA10'], 'Signal'] = 1
df.loc[df['Close'] <= df['MA10'], 'Signal'] = -1

# Calculate trading returns
df['Returns'] = df['Close'].pct_change() * df['Signal'].shift()

# Calculate cumulative returns
df['Cumulative Returns'] = (1 + df['Returns']).cumprod()

# Evaluate trading performance
total_return = df['Cumulative Returns'].iloc[-1] - 1
annualized_return = (1 + total_return) ** (252 / len(df)) - 1
max_drawdown = 1 - df['Cumulative Returns'] / df['Cumulative Returns'].cummax()

print("Total Return:", total_return)
print("Annualized Return:", annualized_return)
print("Max Drawdown:", max_drawdown.min())


Total Return: 1.0332765038704936
Annualized Return: 0.1289905345393596
Max Drawdown: 0.0


In [18]:
## METHOD MA 10 WITH FILTERED DATE
import pandas as pd

# Convert the Date column to string format
df['Date'] = df['Date'].astype(str)

# Filter the dataframe for the desired date range (2022 and 2023)
df_filtered = df[(df['Date'] >= '220101') & (df['Date'] <= '231231')]

# Calculate MA10
df_filtered['MA10'] = df_filtered['Close'].rolling(window=10).mean()

# Define trading signals based on MA10
df_filtered['Signal'] = 0
df_filtered.loc[df_filtered['Close'] > df_filtered['MA10'], 'Signal'] = 1
df_filtered.loc[df_filtered['Close'] <= df_filtered['MA10'], 'Signal'] = -1

# Calculate trading returns
df_filtered['Returns'] = df_filtered['Close'].pct_change() * df_filtered['Signal'].shift()

# Calculate cumulative returns
df_filtered['Cumulative Returns'] = (1 + df_filtered['Returns']).cumprod()

# Calculate points earned per month
df_filtered['YearMonth'] = pd.to_datetime(df_filtered['Date'], format='%y%m%d').dt.to_period('M')
monthly_points = df_filtered.groupby('YearMonth')['Returns'].sum()

print(monthly_points)


YearMonth
2022-01   -0.000545
2022-02   -0.051382
2022-03   -0.002026
2022-04    0.065603
2022-05    0.091553
2022-06    0.006649
2022-07   -0.000394
2022-08    0.004066
2022-09    0.065799
2022-10   -0.033211
2022-11    0.044841
2022-12    0.033062
2023-01    0.033287
2023-02   -0.039758
2023-03   -0.023007
2023-04    0.029752
2023-05    0.004580
2023-06    0.011711
2023-07    0.000179
Freq: M, Name: Returns, dtype: float64


C:\Users\Trieu Pham\AppData\Local\Temp\ipykernel_14108\789332608.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['MA10'] = df_filtered['Close'].rolling(window=10).mean()
C:\Users\Trieu Pham\AppData\Local\Temp\ipykernel_14108\789332608.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Signal'] = 0
C:\Users\Trieu Pham\AppData\Local\Temp\ipykernel_14108\789332608.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [19]:
## OPTIMIZE MA TRADING

import pandas as pd

# Define the range of MA values to test
ma_range = range(5, 51, 1)

# Initialize variables to store the best MA value and corresponding performance metrics
best_ma = None
best_annualized_return = float('-inf')
best_max_drawdown = float('inf')

# Iterate over the MA values
for ma in ma_range:
    # Calculate MA
    df['MA'] = df['Close'].rolling(window=ma).mean()

    # Define trading signals based on MA
    df['Signal'] = 0
    df.loc[df['Close'] > df['MA'], 'Signal'] = 1
    df.loc[df['Close'] <= df['MA'], 'Signal'] = -1

    # Calculate trading returns
    df['Returns'] = df['Close'].pct_change() * df['Signal'].shift()

    # Calculate cumulative returns
    df['Cumulative Returns'] = (1 + df['Returns']).cumprod()

    # Evaluate trading performance
    total_return = df['Cumulative Returns'].iloc[-1] - 1
    annualized_return = (1 + total_return) ** (252 / len(df)) - 1
    max_drawdown = 1 - df['Cumulative Returns'] / df['Cumulative Returns'].cummax()

    # Update the best performance metrics and MA value if necessary
    if annualized_return > best_annualized_return:
        best_ma = ma
        best_annualized_return = annualized_return
        best_max_drawdown = max_drawdown.min()

# Print the results
print("Best MA:", best_ma)
print("Best Annualized Return:", best_annualized_return)
print("Best Max Drawdown:", best_max_drawdown)


Best MA: 8
Best Annualized Return: 0.2629834159695865
Best Max Drawdown: 0.0
